In [27]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date
hoje =  pd.to_datetime(date.today())

In [28]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 17/01/23 - Horário: 14:30:36.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:06.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:10.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:22.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:46.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 17/01/23 - Horário: 14:36:43.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 17/01/23 - Horário: 14:36:54.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 17/01/23 - Horário: 14:37:38.  Portanto não foi baixado novamente.
vrapeelo

In [29]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
ide_data_cols = ['DthEnvio','IdeUsinaOutorga']
ide_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Informações de Usinas

## Carrega banco de dados

In [30]:
cols = ["IdeUsinaOutorga","DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","DatMonitoramento","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado"]
df_usina = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]
df_usina = df_usina[df_usina.IdcUsinaMonitorada == "Sim"].reset_index(drop=True)

In [31]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]
df_usina = pd.merge(df_usina,monitoramentoleilao,how="left",on="IdeUsinaOutorga")

In [32]:
# Carrega BD vrapeelcronograma => rapeel_cronograma
cols = ['IdeUsinaOutorga','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma = rapeel_cronograma.loc[rapeel_cronograma.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
df_usina = pd.merge(df_usina,rapeel_cronograma,how="left",on="IdeUsinaOutorga")

In [33]:
# Carrega BD vrapeelcontratorecurso => mercado
cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)
mercado = mercado.loc[mercado.groupby("IdeUsinaOutorga").DthEnvio.idxmax()]
del mercado['DthEnvio']
df_usina = pd.merge(df_usina,mercado,on="IdeUsinaOutorga",how="left")

In [34]:
# Carrega BD vrapeellicenciamento => ambiental_datas
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del ambiental_datas['DthEnvio']
df_usina = pd.merge(df_usina,ambiental_datas,on="IdeUsinaOutorga",how="left")

In [35]:
# Carrega BD vrapeelacesso => acesso_contratos
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del acesso_contratos['DthEnvio']
df_usina = pd.merge(df_usina,acesso_contratos,on="IdeUsinaOutorga",how="left")

##  Cria colunas com definições e regras

In [36]:
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

df_usina['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    df_usina.loc[df_usina.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = df_usina[validade]
df_usina[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5)

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
992,46582,NaT,2027-10-21,2025-11-06,2027-10-21
688,41902,NaT,NaT,NaT,NaT
1342,48627,NaT,NaT,2031-01-21,2031-01-21
1817,50476,NaT,NaT,NaT,NaT
1607,49642,NaT,NaT,NaT,NaT


In [37]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao

df_usina['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    df_usina.loc[df_usina[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

df_usina.loc[(df_usina.CondicaoAmbiental.isna()) & (df_usina.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
df_usina.loc[(df_usina.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


df_usina[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(df_usina[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

df_usina['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

df_usina.loc[
    ((df_usina.NomSitContratoCCD ==  nsa)&
    (df_usina.NomSitContratoCCT == nsa) &
    df_usina.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

df_usina.loc[
(    ((df_usina.NomSitContratoCCD ==  valido) |
    (df_usina.NomSitContratoCCT == valido)) &
    df_usina.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

df_usina.loc[
    (((df_usina.NomSitContratoCCD ==  vencido) |
    (df_usina.NomSitContratoCCT == vencido) |
    (df_usina.NomSitContratoCCD == nass) |
    (df_usina.NomSitContratoCCT == nass )) &
    df_usina.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(df_usina[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


df_usina['CondicaoUso'] = "Não informado"

df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  nsa)&
    (df_usina.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  valido) |
    (df_usina.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

df_usina.loc[
    (((df_usina.NomSituacaoContratoCUSD ==  vencido) |
    (df_usina.NomSitContratoCUST == vencido) |
    (df_usina.NomSituacaoContratoCUSD == nass) |
    (df_usina.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(df_usina[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


df_usina["PPA"] = "Ambos"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(df_usina[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
0,NaT,2026-03-09,2002-12-11,LI,2026-03-09
1,NaT,NaT,NaT,Sem LP,NaT
2,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
3,NaT,NaT,NaT,Não Informado,NaT
4,NaT,2009-03-10,1999-01-09,LI,2009-03-10
...,...,...,...,...,...
2441,NaT,NaT,NaT,Não Informado,NaT
2442,NaT,NaT,NaT,Não Informado,NaT
2443,NaT,NaT,NaT,Não Informado,NaT
2444,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
711,43147,Não se Aplica,Válido,2023-01-03 15:01:07.533,OK
1906,50954,NaN,NaN,NaT,Não informado
292,35246,Não se Aplica,Não Assinado,2023-01-05 12:18:50.280,Sem Conexão
2324,54909,Não se Aplica,Não se Aplica,2023-01-03 20:02:57.953,Verificar
200,34122,Não se Aplica,Válido,2023-01-02 13:34:47.267,OK
326,35654,Não se Aplica,Não Assinado,2023-01-05 14:44:13.133,Sem Conexão
784,44439,Não se Aplica,Não Assinado,2023-01-02 12:12:49.313,Sem Conexão
1154,47304,Não se Aplica,Válido,2023-01-05 10:09:43.593,OK
1331,48595,Não se Aplica,Não Assinado,2023-01-02 19:21:38.047,Sem Conexão
1816,50475,NaN,NaN,NaT,Não informado


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
2439,70556,NaN,NaN,Não informado
930,45724,Válido,Não se Aplica,OK
485,37770,Não se Aplica,Válido,OK
2281,54665,Não se Aplica,Não se Aplica,Verificar
2295,54776,Não se Aplica,Não Assinado,Sem Conexão
1188,47348,Não se Aplica,Válido,OK
1013,46737,Não se Aplica,Válido,OK
757,44343,NaN,NaN,Não informado
1728,49939,Não se Aplica,Não Assinado,Sem Conexão
1037,46880,Não se Aplica,Não Assinado,Sem Conexão


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
988,46572,Fora do ACR,2021-12-17,ACL
1529,49448,Fora do ACR,NaT,Nenhum
608,38350,Fora do ACR,2021-07-15,ACL
467,37700,ACR,NaT,ACR
2337,55334,Fora do ACR,NaT,Nenhum
823,44539,Fora do ACR,NaT,Nenhum
1392,48916,Fora do ACR,NaT,Nenhum
811,44485,Fora do ACR,NaT,Nenhum
2369,56139,ACR,NaT,ACR
2192,53894,Fora do ACR,NaT,Nenhum


In [38]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

df_usina['criterio'] = np.select(
    [
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


df_usina['dscriterio'] = np.select(
    [
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(df_usina[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
814,44510,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1254,47446,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1326,48590,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
952,46517,Em andamento,Nenhum,LI,OK,0,Usina em Obras em andamento
1831,50534,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2170,53872,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1227,47394,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1299,48518,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
188,33854,Em andamento,Nenhum,LI,OK,0,Usina em Obras em andamento
2067,52134,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"


In [39]:
slp = "Sem LP"
sim = "Sim"
df_usina['criterio_novo'] = np.select(
    [
        (df_usina.IdcSemPrevisao == sim),
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        (df_usina.CondicaoAmbiental == "Sem LP"),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(df_usina[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
1909,50958,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
2212,54140,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2021,51821,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2415,60203,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
1938,51442,Em andamento,ACR,Sem LP,Sem Uso,0,Usina em Obras em andamento


# Informações de UGs

## Carrega banco de dados

In [40]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga",'NumUgUsina',"DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada",'DatLiberOpComerRealizado']
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols].rename(columns={'NumUgUsina':'NumOperacaoUg'})

In [41]:
# Carrega BD vrapeeloperacaoug => ug_rapeel
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ug_rapeel.loc[ug_rapeel.groupby(['IdeUsinaOutorga','NumOperacaoUg']).DthEnvio.idxmax()]

In [42]:
df_ug = pd.merge(df_usina,monitoramentoug,on="IdeUsinaOutorga",how='left')
df_ug = pd.merge(df_ug,ug_rapeel,on=ide_ug,how='left')
df_ug = df_ug.loc[df_ug.DatLiberOpComerRealizado.isna()].reset_index(drop=True)

In [43]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100

In [44]:
cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm','NumUgUsina':'NumOperacaoUg'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)

for col in ['SigTipoGeracao','DatMonitoramento','DatInicioObraOutorgado','DatPrevisaoIniciobra']:
    if col in calculo_previsao.columns:
        calculo_previsao.drop([col],inplace=True,axis=1)

In [45]:
df_ug = pd.merge(df_ug,calculo_previsao,on=ide_ug,how="left")

##  Cria colunas com definições e regras

In [48]:
################################ regra_antiga ################################
df_ug.loc[(df_ug['IdcSemPrevisao'] == "Sim"),'regra_antiga'] = pd.NA
df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] != "OT"),'regra_antiga'] = df_ug[['Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'regra_antiga'] = ( hoje +pd.Timedelta(60,'D'))
df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT"),'regra_antiga'] = df_ug['Previsao_OC']
df_ug.loc[( (df_ug['criterio'] == 1) | (df_ug['criterio'] == 2)) & ( (df_ug['IdcObraParalisada'] == "Sim")),'regra_antiga'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
df_ug.loc[( (df_ug['criterio'] == 1) | (df_ug['criterio'] == 2)) & ( (df_ug['IdcObraParalisada'] == "Não")),'regra_antiga'] = df_ug[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
df_ug.loc[(df_ug['criterio'] >= 3) & ( (df_ug['IdcObraParalisada'] == "Sim")),'regra_antiga'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
df_ug.loc[df_ug['criterio'] >= 3 & ( (df_ug['IdcObraParalisada'] == "Não")),'regra_antiga'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)


################################ regrapmo ################################
df_ug.loc[(df_ug['IdcSemPrevisao'] == "Sim"),'regrapmo'] =  pd.NA
df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] != "OT"),'regrapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'regrapmo'] = ( hoje + pd.Timedelta(60,'D'))
df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT"),'regrapmo'] = df_ug['Previsao_OC']
df_ug.loc[( (df_ug['criterio'] == 8) ),'regrapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
df_ug.loc[( (df_ug['criterio'] == 1) | (df_ug['criterio'] == 2)) & ( (df_ug['IdcObraParalisada'] == "Não")),'regrapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC']].max(axis=1)

df_ug["Dummy"] = hoje + pd.Timedelta(3*365,'D')
df_ug.loc[(df_ug['criterio'] == 3),'regrapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

df_ug["Dummy"] = hoje + pd.Timedelta(4*365,'D')
df_ug.loc[(df_ug['criterio'] == 4),'regrapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

df_ug["Dummy"] = hoje + pd.Timedelta(5*365,'D')
df_ug.loc[(df_ug['criterio'] == 5) | (df_ug['criterio'] == 7),'regrapmo'] =  df_ug[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

df_ug.loc[(df_ug['criterio'] == 6),'regrapmo'] = pd.NA


################################ criterionovopmo ################################
list_condicoes = [
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT")),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT")),
    ((df_ug['criterio_novo'] == 1)),
    ((df_ug['criterio_novo'] == 2)),
    ((df_ug['criterio_novo'] == 3)),
    ((df_ug['criterio_novo'] == 5)),
    ((df_ug['criterio_novo'] == 4)),
    ((df_ug['criterio_novo'] == 6)),
    ((df_ug['criterio_novo'] == 7)),
    ((df_ug['criterio_novo'] == 8)),
    ((df_ug['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

df_ug['criterionovopmo'] = np.select(list_condicoes,list_values)

list_condicoes = [
    ((df_ug['criterio_novo'] == 9)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT")),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT")),
    ((df_ug['criterio_novo'] == 1)),
    ((df_ug['criterio_novo'] == 2)),
    ((df_ug['criterio_novo'] == 3)),
    ((df_ug['criterio_novo'] == 4)),
    ((df_ug['criterio_novo'] == 5)),
    ((df_ug['criterio_novo'] == 7)),
    ((df_ug['criterio_novo'] == 6)),
    ((df_ug['criterio_novo'] == 8))
]


list_values = [
    "Usina Viabilidade Baixa análise da fiscalização",
    "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL",
    "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias",
    "Usina em Teste = Previsão OC conforme data calculada",
    "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras, sem cust, sem PPA e com LI = Previsão OC handicap de 5 anos",
    "Usina sem obras, com cust, sem PPA e sem LI = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem cust, sem PPA e sem LI = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC handicap de 5 anos",
    "Usina sem LP = Previsão OC sem previsão","Usina obras paralisadas == Previsão OC maior entre data RAPEEL e data compromisso"
]


df_ug['dsccriterionovo'] = np.select(list_condicoes,list_values)


################################ regranovapmo ################################
df_ug.loc[(df_ug['IdcSemPrevisao'] == "Sim"),'regranovapmo'] = pd.NA

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT"),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC']].max(axis=1)

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].isna()),'regranovapmo'] = df_ug['DatPrevistaComercial']

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].notna()),'regranovapmo'] = df_ug['Previsao_OC']

df_ug.loc[(df_ug['criterio_novo']== 1) | (df_ug['criterio_novo']== 2) ,'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
df_ug["Dummy"] = hoje + pd.Timedelta(5*365,'D')

df_ug.loc[((df_ug['criterio_novo'] == 3) | (df_ug['criterio_novo'] == 4) | (df_ug['criterio_novo'] == 5) ),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

df_ug.loc[df_ug['criterio_novo']== 6,'regranovapmo'] = pd.NA

df_ug["Dummy"] = hoje + pd.Timedelta(6*365,'D')
df_ug.loc[df_ug['criterio_novo']== 7,'regranovapmo'] =  df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

df_ug.loc[df_ug['criterio_novo']== 8,'regranovapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)


################################ criterio_pmo_ts ################################
list_condicoes = [
    (df_ug['IdcSemPrevisao'] == "Sim"),
    (df_ug['criterio'] == 0) & (df_ug['FASE'] != "OT"),
    (df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),
    (df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT"),
    ((df_ug['PPA'] != "Nenhum")) & ((df_ug['criterio'] == 8) ),
    ( (df_ug['criterio'] == 1)) & ( (df_ug['IdcObraParalisada'] == "Não")),
    ((df_ug['criterio'] == 2)) & ( (df_ug['IdcObraParalisada'] == "Não")),
    ( (df_ug['criterio'] == 8)) & ( (df_ug['PPA'] == "Nenhum") & (df_ug['CondicaoUso'] == "OK")) & ( (df_ug['CondicaoAmbiental'] == "LI") | (df_ug['CondicaoAmbiental'] == "LO") ),
    (df_ug['criterio'] == 3),
    (df_ug['criterio'] == 4),
    (df_ug['criterio'] == 5),
    (df_ug['criterio'] == 7),
    (df_ug['criterio'] == 6),
]

list_values = [8,0.1,0.2,0.3,1.1,1.2,2.2,2.1,3,4,5,7,6]
df_ug['criterio_pmo_ts'] = np.select(list_condicoes,list_values)


################################ dsccriterio_pmo_ts ################################
df_ug.loc[(df_ug['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias == Previsão OC próximos 60 dias"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste == Previsão OC conforme data calculada"

df_ug.loc[((df_ug['PPA'] != "Nenhum")) & ((df_ug['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

df_ug.loc[( (df_ug['criterio'] == 1)) & ( (df_ug['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

df_ug.loc[(df_ug['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste = Previsão OC conforme data calculada"

df_ug.loc[((df_ug['PPA'] != "Nenhum")) & ((df_ug['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

df_ug.loc[( (df_ug['criterio'] == 1)) & ( (df_ug['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

df_ug.loc[((df_ug['criterio'] == 2)) & ( (df_ug['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

df_ug.loc[( (df_ug['criterio'] == 8)) & ( (df_ug['PPA'] == "Nenhum") & (df_ug['CondicaoUso'] == "OK")) & ( (df_ug['CondicaoAmbiental'] == "LI") | (df_ug['CondicaoAmbiental'] == "LO") ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

df_ug.loc[(df_ug['criterio'] == 3),'dsccriterio_pmo_ts'] = "Usina sem PPA com LI sem CUST = Previsão OC maior entre handicap de 3 anos e data RAPEEL e data compromisso e data calculada"

df_ug.loc[(df_ug['criterio'] == 4),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI com CUST = Previsão OC maior entre handicap de 4 anos e data RAPEEL e data compromisso e data calculada"

df_ug.loc[(df_ug['criterio'] == 5),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI sem CUST = Previsão OC maior entre handicap de 5 anos e data RAPEEL e data compromisso e data calculada"

df_ug.loc[(df_ug['criterio'] == 7),'dsccriterio_pmo_ts'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC maior entre handicap de 5 anos e data compromisso e data calculada"

df_ug.loc[(df_ug['criterio'] == 6),'dsccriterio_pmo_ts'] = "Usina sem LP = Previsão OC sem previsão"


################################ criterio_pmo_ts_max_usina ################################
criterio_pmo_ts_max = df_ug.groupby("IdeUsinaOutorga")[['criterio_pmo_ts']].max().rename(columns=
{'criterio_pmo_ts': 'criterio_pmo_ts_max_usina'})
df_ug = df_ug.merge(criterio_pmo_ts_max,on="IdeUsinaOutorga",how='left')


################################ dscjustificativaregra ################################
df_ug.loc[(df_ug['IdcSemPrevisao'] == "Sim"),'dscjustificativaregra'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

df_ug.loc[(df_ug['IdcObraParalisada'] == "Sim") & (df_ug['IdcSemPrevisao'] != "Sim"),'dscjustificativaregra'] = "Paralisação de obras"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 0.2),'dscjustificativaregra'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 0.1) | (df_ug['criterio_pmo_ts'] == 0.3),'dscjustificativaregra'] = "Estágio das obras da usina"


df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 1.1),'dscjustificativaregra'] = "Paralisação de obras"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 1.2),'dscjustificativaregra'] = "Compromisso de venda de energia - PPA"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 2.2),'dscjustificativaregra'] = "Licença de Instalação - LI e acesso a rede - CUST/CUSD"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 3),'dscjustificativaregra'] = "Acesso a rede contratado - CUST/CUSD"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 4),'dscjustificativaregra'] = "Acesso a rede não contratado - CUST/CUSD"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 5),'dscjustificativaregra'] = "Sem Licença de Instalação - LI"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 6),'dscjustificativaregra'] = "Nenhuma licença ambiental válida"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 7),'dscjustificativaregra'] = "Sem RAPEEL"


################################ dscjustificativaregranova ################################

list_condicoes = [
    (df_ug['criterionovopmo'] == 9),
    (df_ug['criterionovopmo'] == 8),
    (df_ug['criterionovopmo'] == 0.2),
    (df_ug['criterionovopmo'] == 0.1) | (df_ug['criterio_pmo_ts'] == 0.3),
    (df_ug['criterionovopmo'] == 1),
    df_ug['criterionovopmo']==2,
    df_ug['criterionovopmo']==3,
    (df_ug.criterio_pmo_ts_max_usina== 4),
    (df_ug.criterio_pmo_ts_max_usina== 5),
    (df_ug.criterio_pmo_ts_max_usina== 6),
    (df_ug.criterio_pmo_ts_max_usina== 7),
]

list_values = [
    "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina",
    "Paralisação de obras",
    "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras",
    "Estágio das obras da usina",
    "Compromisso de venda de energia - PPA",
    "Acesso a rede contratado - CUST/CUSD",
    "Acesso a rede não contratado - CUST/CUSD",
    "Sem Licença de Instalação - LI",
    "Sem Licença de Instalação - LI",
    "Nenhuma licença ambiental válida",
    "Sem RAPEEL"
]

df_ug['dscjustificativaregranova'] = np.select(list_condicoes,list_values)


# Testes